* LSGAN使ってグラタンに似た偽画像を生成する

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# カレントディレクトリの読み込みとカレントディレクトリへの移動
import sys
sys.path.append(f'/content/drive/My Drive/system/')
import os
os.chdir(f'/content/drive/My Drive/system/myanswer')

In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-lzgoawci
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-lzgoawci
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=a50b33ebb1e2a74636fdcdde3bc344429764b8697ea0b94714e13b41050a7596
  Stored in directory: /tmp/pip-ephem-wheel-cache-z42yecqq/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 105kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from glob import glob
from skimage.transform import resize 
import datetime
import pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.misc
import imageio

In [ ]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, is_testing=False):
        if os.path.exists("../pickle/{}_tensor.pickle".format(self.dataset_name)):
            with open("../pickle/{}_tensor.pickle".format(self.dataset_name), 'rb') as p:
                imgs = pickle.load(p)
        else:
            img_pathes = glob('../figure/foodimg128/%s/*.jpg' % (self.dataset_name))
            imgs = []
            for img_path in img_pathes:
                img = self.imread(img_path)
                if not is_testing:
                    img = scipy.misc.imresize(img, self.img_res)
                    if np.random.random() > 0.5:
                        img = np.fliplr(img)
                else:
                    img = scipy.misc.imresize(img, self.img_res)
                imgs.append(img)
            with open('../pickle/{}_tensor.pickle'.format(self.dataset_name), 'wb') as p:
                pickle.dump(imgs , p)

        return np.array(imgs)

    def imread(self, path):
        return scipy.misc.imread(path, mode="RGB").astype(np.float)

In [ ]:
class LSGAN():
    def __init__(self, dataset_name="mnist"):
        self.img_rows = 28
        self.img_cols = 28 
        self.dataset_name = dataset_name
        # 変換させたい画像のデータセットの名前を指定
        if self.dataset_name == "mnist":
            self.channels = 1
        else:
            self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        # model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        # model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        
        if self.dataset_name == "mnist":
            (X_train, _), (_, _) = mnist.load_data()
            # Rescale -1 to 1
            X_train = (X_train.astype(np.float32) - 127.5) / 127.5
            X_train = np.expand_dims(X_train, axis=3)
        else:
            X_train = self.data_loader.load_data()
            # Rescale -1 to 1
            X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                # Plot the progress
                print("epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch+1, d_loss[0], 100*d_loss[1], g_loss))
                self.sample_images(epoch)

    def sample_images(self, epoch):
        os.makedirs('../result/%s/lsgan' % self.dataset_name, exist_ok=True)
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                if self.dataset_name == "mnist":
                    axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                else:
                    axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("../result/{}/lsgan/epoch{}.png".format(self.dataset_name, epoch),
                     transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        plt.close()

In [ ]:
lsgan = LSGAN(dataset_name="gratin")
# lsgan = LSGAN(dataset_name="mnist")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
lsgan.train(epochs=140000, batch_size=32, sample_interval=50000)

epoch 1 [D loss: 1.836925, acc.: 50.00%] [G loss: 0.878412]
epoch 10001 [D loss: 0.049168, acc.: 96.88%] [G loss: 1.000912]
epoch 20001 [D loss: 0.041113, acc.: 98.44%] [G loss: 1.017995]
epoch 30001 [D loss: 0.022450, acc.: 100.00%] [G loss: 1.053429]
epoch 40001 [D loss: 0.027107, acc.: 100.00%] [G loss: 1.074239]
epoch 50001 [D loss: 0.009968, acc.: 100.00%] [G loss: 0.965219]
epoch 60001 [D loss: 0.007064, acc.: 100.00%] [G loss: 0.996799]
epoch 70001 [D loss: 0.024069, acc.: 98.44%] [G loss: 1.019677]
epoch 80001 [D loss: 0.020637, acc.: 100.00%] [G loss: 1.116776]
epoch 90001 [D loss: 0.007198, acc.: 100.00%] [G loss: 0.993552]
epoch 100001 [D loss: 0.014661, acc.: 100.00%] [G loss: 1.104152]
epoch 110001 [D loss: 0.011516, acc.: 98.44%] [G loss: 1.009000]
epoch 120001 [D loss: 0.006608, acc.: 100.00%] [G loss: 1.036598]
epoch 130001 [D loss: 0.037577, acc.: 100.00%] [G loss: 0.913939]
epoch 140001 [D loss: 0.027350, acc.: 100.00%] [G loss: 1.357183]


KeyboardInterrupt: ignored